In [ ]:
import feedparser
from logging import getLogger
import logging
from pprint import pprint as print
from bs4 import BeautifulSoup
from src.util import init_logger
import yaml

In [ ]:
init_logger()
logger = getLogger()

In [ ]:
from openai import ChatCompletion
from root import absolute
import os



In [ ]:
import opml

inoreader_opml_path = absolute("resource", "inoreader-opml.xml")
res = opml.parse(inoreader_opml_path)
with open(inoreader_opml_path, "r") as f:
    outline = opml.parse(f)
for item in outline:
    print(item.text)
    for each in item:
        print(each.text)

In [ ]:
from src.util import CONFIG_PATH
with open("output.yml", "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
    print(config)

In [ ]:
cfg = feedparser.parse("https://rsshub.dcts.top/tophub/0MdKkV0dw1")
print(cfg)

In [2]:
import yaml
import xml.etree.ElementTree as ET

def convert_yaml_to_opml(yaml_path, opml_path):
    with open(yaml_path, 'r') as file:
        data = yaml.safe_load(file)

    
    opml = ET.Element("opml", version="1.0")
    head = ET.SubElement(opml, "head")
    title = ET.SubElement(head, "title")
    title.text = "Feeds of 435212619 from Inoreader [https://www.inoreader.com]"
    
    body = ET.SubElement(opml, "body")

    for category_name, categoty_vals in data.items():
        outline = ET.SubElement(body, 'outline', text=category_name, title=category_name)
        xml_url = 
        for item in categoty_vals:
            feed_outline = ET.SubElement(outline, "outline", text=item['text'], title=item['text'], type='rss',
                                        xmlUrl=item['url'], htmlUrl=item['htmlUrl'])
    
    tree = ET.ElementTree(opml)
    tree.write(opml_path, encoding='utf-8', xml_declaration=True)
    
convert_yaml_to_opml("/Users/wenzhuo/工作/project/2023S3_私有消息聚合/RSS-Master/resource/config.yml", "output.opml")